In [85]:
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
from scipy.sparse import hstack
from sklearn.pipeline import FeatureUnion

In [86]:
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC, LinearSVC
from sklearn import metrics
from sklearn import utils
import sklearn

In [87]:
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC, LinearSVC
from sklearn import metrics
from sklearn import utils
import sklearn
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import xgboost as xgb



In [88]:
from collections import Counter, OrderedDict
import collections

In [89]:
import joblib


In [90]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


In [91]:
import re

In [92]:
from textblob import TextBlob

Based on https://www.kaggle.com/venomsnake/analyzing-sarcasm-comprehensive-eda-detection and https://github.com/surajr/SarcasmDetection

In [93]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [94]:
df_isarcasm=pd.read_csv("/content/drive/MyDrive/ISarcasm/DataSet/train.En.csv")

In [95]:
df_isarcasm=df_isarcasm.loc[df_isarcasm['sarcastic']==1]

In [96]:
df_isarcasm=df_isarcasm[['tweet','sarcasm', 'irony',
       'satire', 'understatement', 'overstatement', 'rhetorical_question']]

In [97]:
train, validate, test = \
              np.split(df_isarcasm.sample(frac=1, random_state=42), 
                       [int(.6*len(df_isarcasm)), int(.8*len(df_isarcasm))])

In [98]:
train_new=pd.concat([train, validate], ignore_index=True)

In [99]:
def count_character_type(text):
    text=str(text)

    specialChar = 0
    for i in range(0, len(text)):
        ch = text[i]
        if ch == "!":
            specialChar+=1
        
    return specialChar
    
def count_question_mark(text):
    text=str(text)
    specialChar = 0
    for i in range(0, len(text)):
        ch = text[i]
        if ch == "?":
            specialChar+=1
        
    return specialChar

The Python NLP library, TextBlob has a method to roughly quantify if a sentence is fact or opinion.¶
The method outputs a number ranging from 0 to 1, an output close to 0 indicates the sentence is highly factual and close to 1 means the sentence is highly subjective. Here, we take the sum for every comment, an overall higher sum then, will be indicative of higher subjectivity

In [100]:
def get_subjectivity(text):
    return TextBlob(str(text)).sentiment.subjectivity

The distribution of the 4 most important parts-of-speech (Noun, Verb, Adjective,Pronoun) in sarcastic and non sarcastic comments might shed some light, so let's plot it as a grouped bar chart

In [101]:
import spacy

In [102]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [103]:
nlp = spacy.load('en_core_web_sm')
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [104]:
def pos_counter(doc):
    verb_count = 0
    adj_count = 0
    pron_count = 0
    noun_count=0
    for tok in doc:
        if tok.pos_ == "VERB":
            verb_count=verb_count+1
        elif tok.pos_ == "ADJ":
            adj_count=adj_count+1
        elif tok.pos_ == "PRON":
            pron_count=pron_count+1
        elif tok.pos_ == "PROPN":
            noun_count=noun_count+1
    return (verb_count,adj_count,pron_count,noun_count)

In [105]:
def get_verb_count(text):
  verb_count = 0
  doc = nlp(str(text))
  for tok in doc:
        if tok.pos_ == "VERB":
            verb_count=verb_count+1
  return verb_count/len(str(text))

def get_noun_count(text):
  noun_count = 0
  doc = nlp(str(text))
  for tok in doc:
        if tok.pos_ == "PROPN":
            noun_count=noun_count+1
  return noun_count/len(str(text))

def get_pron_count(text):
  pron_count = 0
  doc = nlp(str(text))
  for tok in doc:
        if tok.pos_ == "PRON":
            pron_count=pron_count+1
  return pron_count/len(str(text))


def get_adj_count(text):
  adj_count = 0
  doc = nlp(str(text))
  for tok in doc:
        if tok.pos_ == "ADJ":
            adj_count=adj_count+1
  return adj_count/len(str(text))


def named_entity_preprocess(data):
    sentences = nltk.sent_tokenize(data)
    tokenized_sentences = [nltk.word_tokenize(sentence) for sentence in sentences]
    tagged_sentences = [nltk.pos_tag(sentence) for sentence in tokenized_sentences]
    chunked_sentences = nltk.ne_chunk_sents(tagged_sentences, binary=True)
    return chunked_sentences

#identify whether feature is NE or not
def extract_entity_names(t):
    entity_names = []
    if hasattr(t, 'label') and t.label():
        if t.label() == 'NE':
            entity_names.append(' '.join([child[0] for child in t]))
        else:
            for child in t:
                entity_names.extend(extract_entity_names(child))

    return entity_names

#counting of NE via loop and then normalizing it for each tweet
def named_entity_count(tweet):

#     named_entity_list = []
      
#     for tweet in sample:
    tweet=str(tweet)
    chunked_sentences = named_entity_preprocess(tweet)
    entity_names = []
    for tree in chunked_sentences:
        entity_names.extend(extract_entity_names(tree))
    ne_count = len(entity_names)
    word_count = len(tweet.split())
    ne_score = ne_count / word_count
#     named_entity_list.append(ne_score)
    return ne_score

Sarcastic comments tend to have profanity interspersed within, let's get a count of both sarcastic and non sarcastic and see how they compare

In [106]:
!pip install better_profanity 
from better_profanity import profanity

In [107]:
def count_profane_words(text):
    count = 0
    for sent in str(text).split():
        if profanity.contains_profanity(sent) == True:
            count = count+1
    return count

get entity cloud

In [108]:
def entity_wordcloud(text):
    named_entities = []    
    sent = nlp(str(text))
    for ent in sent.ents:
        if ent.label_ == 'PERSON' or 'ORG' or 'GPE':
            named_entities.append(ent.text)
            
    doc = " ".join(named_entities)
    return doc

In [109]:
!git clone https://github.com/larsmans/sentiwordnet.git

fatal: destination path 'sentiwordnet' already exists and is not an empty directory.


In [110]:
import csv

In [111]:
class load_senti_word_net(object):
    """
    constructor to load the file and read the file as CSV
    6 columns - pos, ID, PosScore, NegScore, synsetTerms, gloss
    synsetTerms can have multiple similar words like abducting#1 abducent#1 and will read each one and calculaye the scores
    """
    
    def __init__(self):
        sent_scores = collections.defaultdict(list)
        with io.open("/content/sentiwordnet/SentiWordNet_3.0.0_20130122.txt") as fname:
            file_content = csv.reader(fname, delimiter='\t',quotechar='"')
            
            for line in file_content:                
                if line[0].startswith('#') :
                    continue                    
                pos, ID, PosScore, NegScore, synsetTerms, gloss = line
                for terms in synsetTerms.split(" "):
                    term = terms.split("#")[0]
                    term = term.replace("-","").replace("_","")
                    key = "%s/%s"%(pos,term.split("#")[0])
                    try:
                        sent_scores[key].append((float(PosScore),float(NegScore)))
                    except:
                        sent_scores[key].append((0,0))
                    
        for key, value in sent_scores.items():
            sent_scores[key] = np.mean(value,axis=0)
        
        self.sent_scores = sent_scores    
     
    """
    For a word,
    nltk.pos_tag(["Suraj"])
    [('Suraj', 'NN')]
    """
    
    def score_word(self, word):
        pos = nltk.pos_tag([word])[0][1]
        return self.score(word, pos)
    
    def score(self,word, pos):
        """
        Identify the type of POS, get the score from the senti_scores and return the score
        """
        
        if pos[0:2] == 'NN':
            pos_type = 'n'
        elif pos[0:2] == 'JJ':
            pos_type = 'a'
        elif pos[0:2] =='VB':
            pos_type='v'
        elif pos[0:2] =='RB':
            pos_type = 'r'
        else:
            pos_type =  0
            
        if pos_type != 0 :    
            loc = pos_type+'/'+word
            score = self.sent_scores[loc]
            if len(score)>1:
                return score
            else:
                return np.array([0.0,0.0])
        else:
            return np.array([0.0,0.0])
        
    """
    Repeat the same for a sentence
    nltk.pos_tag(word_tokenize("My name is Suraj"))
    [('My', 'PRP$'), ('name', 'NN'), ('is', 'VBZ'), ('Suraj', 'NNP')]    
    """    
        
    def score_sentencce(self, sentence):
        pos = nltk.pos_tag(sentence)
        # print (pos)
        mean_score = np.array([0.0, 0.0])
        for i in range(len(pos)):
            mean_score += self.score(pos[i][0], pos[i][1])
            
        return mean_score
    
    def pos_vector(self, sentence):
        pos_tag = nltk.pos_tag(sentence)
        vector = np.zeros(4)
        
        for i in range(0, len(pos_tag)):
            pos = pos_tag[i][1]
            if pos[0:2]=='NN':
                vector[0] += 1
            elif pos[0:2] =='JJ':
                vector[1] += 1
            elif pos[0:2] =='VB':
                vector[2] += 1
            elif pos[0:2] == 'RB':
                vector[3] += 1
                
        return vector
            

In [112]:
def gram_features(features,sentence):
    sentence_rep = replace_reg(str(sentence))
    token = nltk.word_tokenize(sentence_rep)
    token = [porter.stem(i.lower()) for i in token]        
    
    bigrams = nltk.bigrams(token)
    bigrams = [tup[0] + ' ' + tup[1] for tup in bigrams]
    grams = token + bigrams
    #print (grams)
    for t in grams:
        features['contains(%s)'%t]=1.0

In [113]:
def pos_features(features,sentence):
    sentence_rep = replace_reg(sentence)
    token = nltk.word_tokenize(sentence_rep)
    token = [ porter.stem(each.lower()) for each in token]
    pos_vector = sentiments.pos_vector(token)
    for j in range(len(pos_vector)):
        features['POS_'+str(j+1)] = pos_vector[j]
    # print ("done")
    

In [114]:
def capitalization(features,sentence):
    count = 0
    for i in range(len(sentence)):
        count += int(sentence[i].isupper())
    features['Capitalization'] = int(count > 3)
    # print (count)

In [115]:
import nltk
import re

#dictionnary to sentiment analysis
emo_repl = {
    #good emotions
    "&lt;3" : " good ",
    ":d" : " good ",
    ":dd" : " good ",
    ":p" : " good ",
    "8)" : " good ",
    ":-)" : " good ",
    ":)" : " good ",
    ";)" : " good ",
    "(-:" : " good ",
    "(:" : " good ",
    
    "yay!" : " good ",
    "yay" : " good ",
    "yaay" : " good ",
    "yaaay" : " good ",
    "yaaaay" : " good ",
    "yaaaaay" : " good ",    
    #bad emotions
    ":/" : " bad ",
    ":&gt;" : " sad ",
    ":')" : " sad ",
    ":-(" : " bad ",
    ":(" : " bad ",
    ":s" : " bad ",
    ":-s" : " bad "
}

#dictionnary for general (i.e. topic modeler)
emo_repl2 = {
    #good emotions
    "&lt;3" : " heart ",
    ":d" : " smile ",
    ":p" : " smile ",
    ":dd" : " smile ",
    "8)" : " smile ",
    ":-)" : " smile ",
    ":)" : " smile ",
    ";)" : " smile ",
    "(-:" : " smile ",
    "(:" : " smile ",
       
    #bad emotions
    ":/" : " worry ",
    ":&gt;" : " angry ",
    ":')" : " sad ",
    ":-(" : " sad ",
    ":(" : " sad ",
    ":s" : " sad ",
    ":-s" : " sad "
}

#general
re_repl = {
    r"\br\b" : "are",
    r"\bu\b" : "you",
    r"\bhaha\b" : "ha",
    r"\bhahaha\b" : "ha",
    r"\bdon't\b" : "do not",
    r"\bdoesn't\b" : "does not",
    r"\bdidn't\b" : "did not",
    r"\bhasn't\b" : "has not",
    r"\bhaven't\b" : "have not",
    r"\bhadn't\b" : "had not",
    r"\bwon't\b" : "will not",
    r"\bwouldn't\b" : "would not",
    r"\bcan't\b" : "can not",
    r"\bcannot\b" : "can not"    
}

emo_repl_order = [k for (k_len,k) in reversed(sorted([(len(k),k) for k in emo_repl.keys()]))]
emo_repl_order2 = [k for (k_len,k) in reversed(sorted([(len(k),k) for k in emo_repl2.keys()]))]

def replace_emo(sentence):
    sentence2 = sentence
    for k in emo_repl_order:
        sentence2 = sentence2.replace(k,emo_repl[k])
    for r, repl in re_repl.items():
        sentence2 = re.sub(r,repl,sentence2)
    return sentence2

def replace_reg(sentence):
    sentence2 = sentence
    for k in emo_repl_order2:
        sentence2 = sentence2.replace(k,emo_repl2[k])
    for r, repl in re_repl.items(): 
        sentence2 = re.sub(r,repl,sentence2)
    return sentence2

In [116]:
from gensim import corpora, models, similarities
import nltk
from nltk.corpus import stopwords
import numpy as np
import pandas as pd

class topic(object):
    def __init__(self, nbtopic = 100, alpha=1,model=None,dicttp=None):
        self.nbtopic = nbtopic
        self.alpha = alpha
        self.porter = nltk.PorterStemmer()
        self.stop = stopwords.words('english')+['.','!','?','"','...','\\',"''",'[',']','~',"'m","'s",';',':','..','$']
        if model!=None and dicttp!=None:
            self.lda = models.ldamodel.LdaModel.load(model)
            self.dictionary =  corpora.Dictionary.load(dicttp)
            
    def fit(self,documents):
        
        documents_mod = documents
        tokens = [nltk.word_tokenize(sentence) for sentence in documents_mod]
        tokens = [[self.porter.stem(t.lower()) for t in sentence if t.lower() not in self.stop] for sentence in tokens]        
            
        self.dictionary = corpora.Dictionary(tokens)
        corpus = [self.dictionary.doc2bow(text) for text in tokens]
        self.lda = models.ldamodel.LdaModel(corpus,id2word=self.dictionary, num_topics=self.nbtopic,alpha=self.alpha)
        
        self.lda.save('topics.tp')
        self.dictionary.save('topics_dict.tp')
        
    def get_topic(self,topic_number):
        
        return self.lda.print_topic(topic_number)
    
    def transform(self,sentence):
        
        sentence_mod = sentence
        tokens = nltk.word_tokenize(sentence_mod)
        tokens = [self.porter.stem(t.lower()) for t in tokens if t.lower() not in self.stop] 
        corpus_sentence = self.dictionary.doc2bow(tokens)
        
        return self.lda[corpus_sentence] 

In [117]:
porter = nltk.PorterStemmer()


In [118]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [119]:
import string
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
def sentiment_extract(features, sentence):
    sentence_rep = replace_reg(sentence)
    token = nltk.word_tokenize(sentence_rep)    
    token = [porter.stem(i.lower()) for i in token]   
    mean_sentiment = sentiments.score_sentencce(token)
    sent_score = sid.polarity_scores(sentence)
    
    features["Positive Sentiment"] = mean_sentiment[0]
    features["Negative Sentiment"] = mean_sentiment[1]
    features["Neutral Sentiment"] = sent_score['neu']
    features["sentiment"] = mean_sentiment[0] - mean_sentiment[1]
    #print(mean_sentiment[0], mean_sentiment[1])
    
    try:
        text = TextBlob(" ".join([""+i if i not in string.punctuation and not i.startswith("'") else i for i in token]).strip())
        features["Blob Polarity"] = text.sentiment.polarity
        features["Blob Subjectivity"] = text.sentiment.subjectivity
        #print (text.sentiment.polarity,text.sentiment.subjectivity )
    except:
        features["Blob Polarity"] = 0
        features["Blob Subjectivity"] = 0
        # print("do nothing")
        
    
    first_half = str(token[0:len(token)//2] )  
    mean_sentiment_half = sentiments.score_sentencce(first_half)
    sent_score_half = sid.polarity_scores(first_half)
    features["positive Sentiment first half"] = mean_sentiment_half[0]
    features["negative Sentiment first half"] = mean_sentiment_half[1]
    features["Neutral Sentiment first half"] = sent_score_half['neu']
    features["first half sentiment"] = mean_sentiment_half[0]-mean_sentiment_half[1]
    try:
        text = TextBlob(" ".join([""+i if i not in string.punctuation and not i.startswith("'") else i for i in first_half]).strip())
        features["first half Blob Polarity"] = text.sentiment.polarity
        features["first half Blob Subjectivity"] = text.sentiment.subjectivity
        #print (text.sentiment.polarity,text.sentiment.subjectivity )
    except:
        features["first Blob Polarity"] = 0
        features["first Blob Subjectivity"] = 0
        # print("do nothing")
    
    second_half = str(token[len(token)//2:])
    mean_sentiment_sechalf = sentiments.score_sentencce(second_half)
    sent_score_sechalf = sid.polarity_scores(second_half)
    features["positive Sentiment second half"] = mean_sentiment_sechalf[0]
    features["negative Sentiment second half"] = mean_sentiment_sechalf[1]
    features["Neutral Sentiment second half"] = sent_score_sechalf['neu']
    features["second half sentiment"] = mean_sentiment_sechalf[0]-mean_sentiment_sechalf[1]
    try:
        text = TextBlob(" ".join([""+i if i not in string.punctuation and not i.startswith("'") else i for i in second_half]).strip())
        features["second half Blob Polarity"] = text.sentiment.polarity
        features["second half Blob Subjectivity"] = text.sentiment.subjectivity
        #print (text.sentiment.polarity,text.sentiment.subjectivity )
    except:
        features["second Blob Polarity"] = 0
        features["second Blob Subjectivity"] = 0
        # print("do nothing")  
    

In [120]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [121]:
topic_mod = topic(nbtopic=200,alpha='symmetric')
topic_mod.fit(train_new['tweet'].values)
# topic_mod = topic(model=os.path.join('topics.tp'),dicttp=os.path.join('topics_dict.tp'))
def topic_feature(features,sentence,topic_modeler):    
    topics = topic_modeler.transform(sentence)    
    for j in range(len(topics)):
        features['Topic :'] = topics[j][1]

In [122]:
!pip install emoji

In [123]:
from emoji import UNICODE_EMOJI



In [124]:
"""
Function to extract the below features.
1. Presence of emoji's
2. Count of number of Question marks
3. Count of number of Exclamation marks
4. presence of hashtags other than #sarcasm
5. presence of any @user tweets

"""
from collections import defaultdict

def Emoji_present(text):
    emoji = {
    "&lt;3" : "positive",":D" : "positive",	":d" : "positive", ":dd" : "positive", ":P" : "positive", ":p" : "positive","8)" : "positive",
    "8-)" : "positive",  ":-)" : "positive",    ":)" : "positive",    ";)" : "positive",    "(-:" : "positive",    "(:" : "positive",
    ":')" : "positive",    "xD" : "positive",    "XD" : "positive",  "yay!" : "positive",  "yay" : "positive",  "yaay" : "positive",
    "yaaay" : "positive",  "yaaaay" : "positive", "yaaaaay" : "positive", "Yay!" : "positive", "Yay" : "positive", "Yaay" : "positive",
    "Yaaay" : "positive", "Yaaaay" : "positive", "Yaaaaay" : "positive",  ":/" : "negative", "&gt;" : "negative", ":'(" : "negative",
    ":-(" : "negative", ":(" : "negative", ":s" : "negative",":-s" : "negative","-_-" : "negative", "-.-" : "negative" }
    emoji_count =defaultdict(int) 
    isPresent=False    
        
    for word in str(text):
        if word in emoji:            
            isPresent = True

    
    count = 0
    for emoji in UNICODE_EMOJI:
        count += str(text).count(emoji)
        if count>=1:
          isPresent=True      

    
    return isPresent 



In [125]:
import io

sentiments = load_senti_word_net()

def get_features(sentence, topic_modeler):
    features = {}
    # gram_features(features,sentence)
    pos_features(features,sentence)
    sentiment_extract(features, sentence)
    capitalization(features,sentence)
    topic_feature(features, sentence,topic_modeler)
    return features

In [126]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [127]:
train_new.dropna(inplace=True)

In [128]:
train_new

,tweet,sarcasm,irony,satire,understatement,overstatement,rhetorical_question
0,All the shade i have been hearing about Ben Pl...,1.0,0.0,0.0,0.0,0.0,0.0
1,you would think the odds of me sitting next to...,1.0,0.0,0.0,0.0,0.0,0.0
2,Billy Gunn sorta relevant for the first time i...,1.0,0.0,0.0,1.0,0.0,0.0
3,"I see Dettol, Toilet Duck and Zoflora are all ...",1.0,0.0,0.0,0.0,0.0,1.0
4,@catboychika Aren't you happy I gave you less ...,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
688,omg my ships are gonna love what taylor swift ...,1.0,0.0,0.0,0.0,0.0,0.0
689,Maybe if I watched suits in year 12 I’d be stu...,1.0,0.0,0.0,0.0,0.0,0.0
690,"Me: ""Did you you find everything you were look...",1.0,0.0,0.0,0.0,0.0,1.0
691,Broke: doing assignments in order of due date\...,1.0,0.0,0.0,0.0,0.0,0.0


In [129]:
import re
featureset=[]
for i, row in train_new.iterrows():
    # if i==2774or i==2775:
    #   continue
    temp = str(train_new["tweet"][i])
    temp = re.sub(r'[^\x00-\x7F]+','',temp)
    featureset.append((get_features(temp,topic_mod), train_new[['sarcasm', 'irony',
       'satire', 'understatement', 'overstatement', 'rhetorical_question']].loc[i].values, train_new["tweet"][i]))

In [130]:
c = []
for i in range(0,len(featureset)):
    c.append(pd.DataFrame(featureset[i][0],index=[i]))
result = pd.concat(c)

In [131]:
result.insert(loc=0,column="label",value='0')
for i in range(0, len(featureset)):
    result["label"].loc[i] = featureset[i][1]   

result.insert(loc=0,column="text",value='0')
for i in range(0, len(featureset)):
    result["text"].loc[i] = featureset[i][2]  
    

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [132]:
result.head()

,text,label,POS_1,POS_2,POS_3,POS_4,Positive Sentiment,Negative Sentiment,Neutral Sentiment,sentiment,Blob Polarity,Blob Subjectivity,positive Sentiment first half,negative Sentiment first half,Neutral Sentiment first half,first half sentiment,first half Blob Polarity,first half Blob Subjectivity,positive Sentiment second half,negative Sentiment second half,Neutral Sentiment second half,second half sentiment,second half Blob Polarity,second half Blob Subjectivity,Capitalization,Topic :
0,All the shade i have been hearing about Ben Pl...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",16.0,3.0,8.0,0.0,0.486475,0.366756,0.932,0.119719,0.200000,0.540000,1.133333,0.416667,1.0,0.716667,0.0,0.0,0.491667,0.333333,1.0,0.158333,0.0,0.0,1,0.956739
1,you would think the odds of me sitting next to...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",9.0,6.0,5.0,0.0,0.617476,0.491941,0.772,0.125535,0.020833,0.156250,0.725000,0.450000,1.0,0.275000,0.0,0.0,0.150000,0.100000,1.0,0.050000,0.0,0.0,0,0.947632
2,Billy Gunn sorta relevant for the first time i...,"[1.0, 0.0, 0.0, 1.0, 0.0, 0.0]",6.0,1.0,0.0,0.0,0.229167,0.000000,1.000,0.229167,0.250000,0.333333,0.150000,0.000000,1.0,0.150000,0.0,0.0,0.150000,0.050000,1.0,0.100000,0.0,0.0,0,0.875625
3,"I see Dettol, Toilet Duck and Zoflora are all ...","[1.0, 0.0, 0.0, 0.0, 0.0, 1.0]",7.0,0.0,5.0,1.0,0.111435,0.109261,1.000,0.002175,-0.033333,0.000000,0.616667,0.333333,1.0,0.283333,0.0,0.0,0.266667,0.133333,1.0,0.133333,0.0,0.0,1,0.917083
4,@catboychika Aren't you happy I gave you less ...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",5.0,1.0,3.0,1.0,0.041667,0.316667,0.750,-0.275000,-0.483333,0.483333,0.075000,0.350000,1.0,-0.275000,0.0,0.0,0.225000,0.000000,1.0,0.225000,0.0,0.0,0,0.889444


In [133]:
result['contain_emoji']=result['text'].apply(Emoji_present)

In [134]:
result['count_special_chracter']=result['text'].apply(count_character_type)

In [135]:
result['count_question_marks']=result['text'].apply(count_question_mark)

In [136]:
result['subjectivity']=result['text'].apply(get_subjectivity)

In [137]:
result['count_verbs']=result['text'].apply(get_verb_count)

In [138]:
result['count_nouns']=result['text'].apply(get_noun_count)

In [139]:
result['count_pronun']=result['text'].apply(get_pron_count)

In [140]:
result['count_adjct']=result['text'].apply(get_adj_count)

In [141]:
result['entities']=result['text'].apply(entity_wordcloud)

In [142]:
result['count_profane']=result['text'].apply(count_profane_words)

In [143]:
def replace_hashtag_emoji(tweet):
    tweet=str(tweet)
    tweet = re.sub(r'\B#','', tweet)
    tweet = emoji.demojize(tweet, delimiters=("", ""))
    return tweet

In [144]:
def has_irony_hashtag(text):
        text=str(text)
        regex = re.compile("#not[\W]*$|#not\\s+?#|#not\\s*?\\bhttp\\b|#irony|#sarcasm|#fake|#naah")
        if regex.search(text.lower()):
            return 1.0
        return 0.0

In [145]:
# ekphrasis-based features (extracted from pre-processed data)

tags =  ['<allcaps>', '<annoyed>', '<censored>', '<date>', '<elongated>', '<emphasis>', '<happy>',
         '<hashtag>', '<heart>', '<kiss>', '<laugh>', '<money>', '<number>', '<percent>', '<phone>',
         '<repeated>', '<sad>', '<shocking>', '<surprise>', '<time>', '<tong>', '<url>', '<user>',
         '<wink>']
def chunkIt(seq, n):
    """splits the list into n approximately equal sub-lists. source: goo.gl/VrHKeR"""
    avg = len(seq) / float(n)
    out = []
    last = 0.0
    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    return out

def tweet_vecs(twt, n=2):
    """extract a feature vector for a single tweet, based on the counts of the annotation tags
        split the tweet to n equal parts and computes the same features for each part"""
    twt = twt.split()
    chunks = chunkIt(twt, n)
    
    scores = []
    
    for chunk in chunks:
        for tag in tags:
            scores.append(sum(1 for t in chunk if t == tag))
    return scores
    
def feats(text):
    """apply the tweet_vecs function on all tweets and return a result in a list"""
    text=str(text)
    return [tweet_vecs(twt) for twt in text]

In [146]:
!pip install demoji

In [147]:
import demoji
demoji.download_codes()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  


In [148]:
import string
count_punc = lambda l1, l2: len(list(filter(lambda c: c in l2, l1)))

In [149]:
nltk.download('maxent_ne_chunker')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!


True

In [150]:
nltk.download('words')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [151]:
import emoji

In [152]:
result['capitalisation_rate'] = result['text'].apply(lambda x: len(list(ch for ch in x if ch.isupper() == 1))/len(x))
result['hashtag'] = result['text'].apply(lambda x: re.findall(r'\B#\w*[a-zA-Z]+\w*', x))
result['hashtag_counts'] = result['text'].apply(lambda x: len(re.findall(r'\B#\w*[a-zA-Z]+\w*', x)))
result['length'] = result['text'].apply(lambda x: len(x.split(' ')))
result['hashtag_word_ratio'] = result['hashtag_counts']/result['length']

result['# punctuation'] = result['text'].apply(lambda x: count_punc(x, string.punctuation))
result['NE_score']=result['text'].apply(named_entity_count)
result['demoji_text']=result['text'].apply(replace_hashtag_emoji)
result['url'] = result['text'].apply(lambda x: 1 if len(re.findall(r'http\S+', x)) else 0)
result['emoji'] = result['text'].apply(lambda x: demoji.findall(x))
result['emoji_counts'] = result['text'].apply(lambda x: len(demoji.findall(x)))
result['@_counts'] = result['text'].apply(lambda x: len(re.findall(r'\B@\w*[a-zA-Z]+\w*', x)))
result['@_counts_rates'] = result['text'].apply(lambda x: len(re.findall(r'\B@\w*[a-zA-Z]+\w*', x))/len(x))
result['irony_hashtag'] = result['text'].apply(has_irony_hashtag)
result['punc_flooding'] = result['text'].apply(lambda x: 1 if len(re.findall(r'([.,/#!$%^&*;:{}=_`~()-])[.,/#!$%^&*;:{}=_`~()-]+', x)) else 0)
result['sarcasm_flag'] = np.where(result['text'].str.contains('#sarcasm'),1,0)
result['hypocrisy_flag'] = np.where(result['text'].str.contains('#hypocrisy'),1,0)
result['seriously_flag'] = np.where(result['text'].str.contains('#seriously'),1,0)
result['not_flag'] = np.where(result['text'].str.contains('#not'),1,0)
result['sar_flag'] = np.where(result['text'].str.contains('$$SAR$$'),1,0)
result['haha_flag'] = np.where(result['text'].str.contains('haha'),1,0)
result['ekphrasis']=result['text'].apply(feats)

In [153]:
result.head()

,text,label,POS_1,POS_2,POS_3,POS_4,Positive Sentiment,Negative Sentiment,Neutral Sentiment,sentiment,Blob Polarity,Blob Subjectivity,positive Sentiment first half,negative Sentiment first half,Neutral Sentiment first half,first half sentiment,first half Blob Polarity,first half Blob Subjectivity,positive Sentiment second half,negative Sentiment second half,Neutral Sentiment second half,second half sentiment,second half Blob Polarity,second half Blob Subjectivity,Capitalization,Topic :,contain_emoji,count_special_chracter,count_question_marks,subjectivity,count_verbs,count_nouns,count_pronun,count_adjct,entities,count_profane,capitalisation_rate,hashtag,hashtag_counts,length,hashtag_word_ratio,# punctuation,NE_score,demoji_text,url,emoji,emoji_counts,@_counts,@_counts_rates,irony_hashtag,punc_flooding,sarcasm_flag,hypocrisy_flag,seriously_flag,not_flag,sar_flag,haha_flag,ekphrasis
0,All the shade i have been hearing about Ben Pl...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",16.0,3.0,8.0,0.0,0.486475,0.366756,0.932,0.119719,0.200000,0.540000,1.133333,0.416667,1.0,0.716667,0.0,0.0,0.491667,0.333333,1.0,0.158333,0.0,0.0,1,0.956739,True,1,3,0.680000,0.008547,0.017094,0.017094,0.008547,Ben Platt @DearEvanHansen Grease 1,0,0.038462,[],0,43,0.000000,6,0.075000,All the shade i have been hearing about Ben Pl...,0,{},0,1,0.004274,0.0,0,0,0,0,0,0,0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,you would think the odds of me sitting next to...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",9.0,6.0,5.0,0.0,0.617476,0.491941,0.772,0.125535,0.020833,0.156250,0.725000,0.450000,1.0,0.275000,0.0,0.0,0.150000,0.100000,1.0,0.050000,0.0,0.0,0,0.947632,True,0,0,0.451389,0.027211,0.000000,0.020408,0.034014,three,1,0.006803,[],0,32,0.000000,1,0.000000,you would think the odds of me sitting next to...,0,{},0,0,0.000000,0.0,0,0,0,0,0,0,0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,Billy Gunn sorta relevant for the first time i...,"[1.0, 0.0, 0.0, 1.0, 0.0, 0.0]",6.0,1.0,0.0,0.0,0.229167,0.000000,1.000,0.229167,0.250000,0.333333,0.150000,0.000000,1.0,0.150000,0.0,0.0,0.150000,0.050000,1.0,0.100000,0.0,0.0,0,0.875625,False,0,0,0.616667,0.000000,0.035714,0.000000,0.035714,Billy Gunn first years,0,0.035714,[],0,10,0.000000,3,0.100000,Billy Gunn sorta relevant for the first time i...,0,{},0,0,0.000000,0.0,1,0,0,0,0,0,0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,"I see Dettol, Toilet Duck and Zoflora are all ...","[1.0, 0.0, 0.0, 0.0, 0.0, 1.0]",7.0,0.0,5.0,1.0,0.111435,0.109261,1.000,0.002175,-0.033333,0.000000,0.616667,0.333333,1.0,0.283333,0.0,0.0,0.266667,0.133333,1.0,0.133333,0.0,0.0,1,0.917083,True,0,0,0.000000,0.027027,0.045045,0.009009,0.000000,Dettol Toilet Duck Zoflora,0,0.090090,[#TrumpPressBriefing],1,18,0.055556,6,0.166667,"I see Dettol, Toilet Duck and Zoflora are all ...",0,{},0,0,0.000000,0.0,1,0,0,0,0,0,0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,@catboychika Aren't you happy I gave you less ...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",5.0,1.0,3.0,1.0,0.041667,0.316667,0.750,-0.275000,-0.483333,0.483333,0.075000,0.350000,1.0,-0.275000,0.0,0.0,0.225000,0.000000,1.0,0.225000,0.0,0.0,0,0.889444,True,0,0,0.533333,0.028986,0.000000,0.043478,0.028986,,0,0.028986,[],0,11,0.000000,2,0.000000,@catboychika Aren't you happy I gave you less ...,0,{},0,1,0.014493,0.0,0,0,0,0,0,0,0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [154]:
len(test)

174

In [155]:
import re
feature_set_test=[]
for i, row in test.iterrows():
    # if i==0:
    #   continue
    temp = str(test["tweet"][i])
    temp = re.sub(r'[^\x00-\x7F]+','',temp)
    feature_set_test.append((get_features(temp,topic_mod), test[['sarcasm', 'irony',
       'satire', 'understatement', 'overstatement', 'rhetorical_question']].loc[i].values,test['tweet'][i]))

In [156]:
c = []
for i in range(0,len(feature_set_test)):
    c.append(pd.DataFrame(feature_set_test[i][0],index=[i]))
result_test = pd.concat(c)

In [157]:
result_test.insert(loc=0,column="label",value='0')
for i in range(0, len(feature_set_test)):
    result_test["label"].loc[i] = feature_set_test[i][1]   

result_test.insert(loc=0,column="text",value='0')
for i in range(0, len(feature_set_test)):
    result_test["text"].loc[i] = feature_set_test[i][2]   
    
    

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [158]:
result_test.head()

,text,label,POS_1,POS_2,POS_3,POS_4,Positive Sentiment,Negative Sentiment,Neutral Sentiment,sentiment,Blob Polarity,Blob Subjectivity,positive Sentiment first half,negative Sentiment first half,Neutral Sentiment first half,first half sentiment,first half Blob Polarity,first half Blob Subjectivity,positive Sentiment second half,negative Sentiment second half,Neutral Sentiment second half,second half sentiment,second half Blob Polarity,second half Blob Subjectivity,Capitalization,Topic :
0,can't wait til a bunch of pudgy suburban dads ...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",4.0,2.0,3.0,1.0,0.095455,0.871212,0.802,-0.775758,0.00,0.000000,0.041667,0.383333,1.0,-0.341667,0.0,0.0,0.425000,0.400000,1.0,0.025000,0.0,0.0,0,0.111667
1,Why are the side effects of antidepressants so...,"[0.0, 1.0, 0.0, 0.0, 1.0, 0.0]",2.0,2.0,1.0,0.0,0.000000,0.104167,0.660,-0.104167,0.00,0.000000,0.225000,0.100000,1.0,0.125000,0.0,0.0,0.150000,0.000000,1.0,0.150000,0.0,0.0,0,0.502500
2,I love to pick my modules on a first come firs...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",9.0,5.0,5.0,1.0,1.166667,0.375000,0.793,0.791667,0.25,0.347917,0.491667,0.133333,1.0,0.358333,0.0,0.0,0.450000,0.100000,1.0,0.350000,0.0,0.0,0,0.249147
3,@RealCandaceO Ok so when does the anger and bi...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",6.0,3.0,1.0,2.0,0.491815,1.409226,0.583,-0.917411,-0.10,0.400000,0.500000,0.350000,1.0,0.150000,0.0,0.0,0.225000,0.000000,1.0,0.225000,0.0,0.0,1,0.390627
4,@AvatarOdin @aubviouslynot sounds like the ame...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",4.0,2.0,1.0,1.0,0.208333,0.083333,0.571,0.125000,0.20,0.200000,0.083333,0.166667,1.0,-0.083333,0.0,0.0,0.266667,0.133333,1.0,0.133333,0.0,0.0,0,0.205925


In [159]:
result_test['contain_emoji']=result_test['text'].apply(Emoji_present)

In [160]:
result_test['count_special_chracter']=result_test['text'].apply(count_character_type)

In [161]:
result_test['count_question_marks']=result_test['text'].apply(count_question_mark)

In [162]:
result_test['subjectivity']=result_test['text'].apply(get_subjectivity)

In [163]:
result_test['count_verbs']=result_test['text'].apply(get_verb_count)

In [164]:
result_test['count_nouns']=result_test['text'].apply(get_noun_count)

In [165]:
result_test['count_pronun']=result_test['text'].apply(get_pron_count)

In [166]:
result_test['count_adjct']=result_test['text'].apply(get_adj_count)

In [167]:
result_test['entities']=result_test['text'].apply(entity_wordcloud)

In [168]:
result_test['count_profane']=result_test['text'].apply(count_profane_words)

In [169]:
result_test['capitalisation_rate'] = result_test['text'].apply(lambda x: len(list(ch for ch in str(x) if ch.isupper() == 1))/len(str(x)))
result_test['hashtag'] = result_test['text'].apply(lambda x: re.findall(r'\B#\w*[a-zA-Z]+\w*', str(x)))
result_test['hashtag_counts'] = result_test['text'].apply(lambda x: len(re.findall(r'\B#\w*[a-zA-Z]+\w*', str(x))))
result_test['length'] = result_test['text'].apply(lambda x: len(str(x).split(' ')))
result_test['hashtag_word_ratio'] = result_test['hashtag_counts']/result_test['length']

result_test['# punctuation'] = result_test['text'].apply(lambda x: count_punc(str(x), string.punctuation))
result_test['NE_score']=result_test['text'].apply(named_entity_count)
result_test['demoji_text']=result_test['text'].apply(replace_hashtag_emoji)
result_test['url'] = result_test['text'].apply(lambda x: 1 if len(re.findall(r'http\S+', str(x))) else 0)
result_test['emoji'] = result_test['text'].apply(lambda x: demoji.findall(str(x)))
result_test['emoji_counts'] = result_test['text'].apply(lambda x: len(demoji.findall(str(x))))
result_test['@_counts'] = result_test['text'].apply(lambda x: len(re.findall(r'\B@\w*[a-zA-Z]+\w*', str(x))))
result_test['@_counts_rates'] = result_test['text'].apply(lambda x: len(re.findall(r'\B@\w*[a-zA-Z]+\w*', str(x)))/len(str(x)))
result_test['irony_hashtag'] = result_test['text'].apply(has_irony_hashtag)
result_test['punc_flooding'] = result_test['text'].apply(lambda x: 1 if len(re.findall(r'([.,/#!$%^&*;:{}=_`~()-])[.,/#!$%^&*;:{}=_`~()-]+', str(x))) else 0)
result_test['sarcasm_flag'] = np.where(result_test['text'].str.contains('#sarcasm'),1,0)
result_test['hypocrisy_flag'] = np.where(result_test['text'].str.contains('#hypocrisy'),1,0)
result_test['seriously_flag'] = np.where(result_test['text'].str.contains('#seriously'),1,0)
result_test['not_flag'] = np.where(result_test['text'].str.contains('#not'),1,0)
result_test['sar_flag'] = np.where(result_test['text'].str.contains('$$SAR$$'),1,0)
result_test['haha_flag'] = np.where(result_test['text'].str.contains('haha'),1,0)
result_test['ekphrasis']=result_test['text'].apply(feats)

In [170]:
result_test.head(20)

,text,label,POS_1,POS_2,POS_3,POS_4,Positive Sentiment,Negative Sentiment,Neutral Sentiment,sentiment,Blob Polarity,Blob Subjectivity,positive Sentiment first half,negative Sentiment first half,Neutral Sentiment first half,first half sentiment,first half Blob Polarity,first half Blob Subjectivity,positive Sentiment second half,negative Sentiment second half,Neutral Sentiment second half,second half sentiment,second half Blob Polarity,second half Blob Subjectivity,Capitalization,Topic :,contain_emoji,count_special_chracter,count_question_marks,subjectivity,count_verbs,count_nouns,count_pronun,count_adjct,entities,count_profane,capitalisation_rate,hashtag,hashtag_counts,length,hashtag_word_ratio,# punctuation,NE_score,demoji_text,url,emoji,emoji_counts,@_counts,@_counts_rates,irony_hashtag,punc_flooding,sarcasm_flag,hypocrisy_flag,seriously_flag,not_flag,sar_flag,haha_flag,ekphrasis
0,can't wait til a bunch of pudgy suburban dads ...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",4.0,2.0,3.0,1.0,0.095455,0.871212,0.802,-0.775758,0.000000,0.000000,0.041667,0.383333,1.0,-0.341667,0.0,0.0,0.425000,0.400000,1.0,0.025000,0.0,0.0,0,0.111667,True,0,0,0.000000,0.038462,0.000000,0.000000,0.012821,the day,0,0.000000,[],0,15,0.000000,1,0.000000,can't wait til a bunch of pudgy suburban dads ...,0,{},0,0,0.000000,0.0,0,0,0,0,0,0,0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,Why are the side effects of antidepressants so...,"[0.0, 1.0, 0.0, 0.0, 1.0, 0.0]",2.0,2.0,1.0,0.0,0.000000,0.104167,0.660,-0.104167,0.000000,0.000000,0.225000,0.100000,1.0,0.125000,0.0,0.0,0.150000,0.000000,1.0,0.150000,0.0,0.0,0,0.502500,True,0,0,0.000000,0.018519,0.000000,0.000000,0.037037,,0,0.018519,[],0,9,0.000000,0,0.000000,Why are the side effects of antidepressants so...,0,{'✨': 'sparkles'},1,0,0.000000,0.0,0,0,0,0,0,0,0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,I love to pick my modules on a first come firs...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",9.0,5.0,5.0,1.0,1.166667,0.375000,0.793,0.791667,0.250000,0.347917,0.491667,0.133333,1.0,0.358333,0.0,0.0,0.450000,0.100000,1.0,0.350000,0.0,0.0,0,0.249147,True,0,0,0.502778,0.023392,0.005848,0.011696,0.023392,first first my entire year 1995,0,0.011696,[],0,35,0.000000,1,0.000000,I love to pick my modules on a first come firs...,0,{'❤️': 'red heart'},1,0,0.000000,0.0,0,0,0,0,0,0,0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,@RealCandaceO Ok so when does the anger and bi...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",6.0,3.0,1.0,2.0,0.491815,1.409226,0.583,-0.917411,-0.100000,0.400000,0.500000,0.350000,1.0,0.150000,0.0,0.0,0.225000,0.000000,1.0,0.225000,0.0,0.0,1,0.390627,True,0,1,0.350000,0.032258,0.000000,0.010753,0.000000,,0,0.064516,[],0,16,0.000000,2,0.062500,@RealCandaceO Ok so when does the anger and bi...,0,{},0,1,0.010753,0.0,0,0,0,0,0,0,0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,@AvatarOdin @aubviouslynot sounds like the ame...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",4.0,2.0,1.0,1.0,0.208333,0.083333,0.571,0.125000,0.200000,0.200000,0.083333,0.166667,1.0,-0.083333,0.0,0.0,0.266667,0.133333,1.0,0.133333,0.0,0.0,0,0.205925,False,0,0,0.000000,0.016393,0.032787,0.000000,0.016393,american,0,0.032787,[],0,8,0.000000,2,0.125000,@AvatarOdin @aubviouslynot sounds like the ame...,0,{},0,2,0.032787,0.0,0,0,0,0,0,0,0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
5,"A whole 2% off for a single order, rejoice","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",3.0,2.0,0.0,0.0,0.191667,0.025000,0.674,0.166667,0.200000,0.400000,0.075000,0.050000,1.0,0.025000,0.0,0.0,0.191667,0.083333,1.0,0.108333,0.0,0.0,0,0.286429,True,0,0,0.307143,0.023810,0.000000,0.000000,0.047619,2%,0,0.023810,[],0,9,0.000000,2,0.000000,"A whole 2% off for a single order, rejoice",0,{},0,0,0.000000,0.0,0,0,0,0,0,0,0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
6,absolutely love waking up to the fire alarm at...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",4.0,0.0,1.0,2.0,0.478472,0.254167,0.430,0.224306,0.500000,0.600000,0.150000,0.000000,1.0,0.150000,0.0,0.0,0.1

In [171]:
result_test.columns

Index(['text', 'label', 'POS_1', 'POS_2', 'POS_3', 'POS_4',
       'Positive Sentiment', 'Negative Sentiment', 'Neutral Sentiment',
       'sentiment', 'Blob Polarity', 'Blob Subjectivity',
       'positive Sentiment first half', 'negative Sentiment first half',
       'Neutral Sentiment first half', 'first half sentiment',
       'first half Blob Polarity', 'first half Blob Subjectivity',
       'positive Sentiment second half', 'negative Sentiment second half',
       'Neutral Sentiment second half', 'second half sentiment',
       'second half Blob Polarity', 'second half Blob Subjectivity',
       'Capitalization', 'Topic :', 'contain_emoji', 'count_special_chracter',
       'count_question_marks', 'subjectivity', 'count_verbs', 'count_nouns',
       'count_pronun', 'count_adjct', 'entities', 'count_profane',
       'capitalisation_rate', 'hashtag', 'hashtag_counts', 'length',
       'hashtag_word_ratio', '# punctuation', 'NE_score', 'demoji_text', 'url',
       'emoji', 'emoji_cou

In [172]:
result['haha_flag'].unique()

array([0, 1])

In [173]:
result_test['haha_flag'].unique()

array([0, 1])

In [174]:
result_test.dropna(inplace=True)

In [175]:
result.dropna(inplace=True)

In [176]:
x_train,y_train=result[['POS_1', 'POS_2', 'POS_3', 'POS_4',
       'Positive Sentiment', 'Negative Sentiment', 'Neutral Sentiment',
       'sentiment', 'Blob Polarity', 'Blob Subjectivity',
       'positive Sentiment first half', 'negative Sentiment first half',
       'Neutral Sentiment first half', 'first half sentiment',
       'first half Blob Polarity', 'first half Blob Subjectivity',
       'positive Sentiment second half', 'negative Sentiment second half',
       'Neutral Sentiment second half', 'second half sentiment',
       'second half Blob Polarity', 'second half Blob Subjectivity',
       'Capitalization', 'Topic :', 'contain_emoji', 'count_special_chracter',
       'count_question_marks', 'subjectivity', 'count_verbs', 'count_nouns',
       'count_pronun', 'count_adjct',  'count_profane',
       'capitalisation_rate',  'hashtag_counts', 
       'hashtag_word_ratio', '# punctuation', 'NE_score', 
        'emoji_counts', '@_counts', '@_counts_rates', 
       'punc_flooding', 'sarcasm_flag', 'hypocrisy_flag', 'seriously_flag',
       'not_flag', 'sar_flag', 'haha_flag']],result[['label']]

In [177]:
x_test,y_test=result_test[['POS_1', 'POS_2', 'POS_3', 'POS_4',
       'Positive Sentiment', 'Negative Sentiment', 'Neutral Sentiment',
       'sentiment', 'Blob Polarity', 'Blob Subjectivity',
       'positive Sentiment first half', 'negative Sentiment first half',
       'Neutral Sentiment first half', 'first half sentiment',
       'first half Blob Polarity', 'first half Blob Subjectivity',
       'positive Sentiment second half', 'negative Sentiment second half',
       'Neutral Sentiment second half', 'second half sentiment',
       'second half Blob Polarity', 'second half Blob Subjectivity',
       'Capitalization', 'Topic :', 'contain_emoji', 'count_special_chracter',
       'count_question_marks', 'subjectivity', 'count_verbs', 'count_nouns',
       'count_pronun', 'count_adjct',  'count_profane',
       'capitalisation_rate',  'hashtag_counts', 
       'hashtag_word_ratio', '# punctuation', 'NE_score', 
        'emoji_counts', '@_counts', '@_counts_rates', 
       'punc_flooding', 'sarcasm_flag', 'hypocrisy_flag', 'seriously_flag',
       'not_flag', 'sar_flag', 'haha_flag']],result_test[['label']]

In [178]:
from sklearn.multioutput import MultiOutputClassifier

In [179]:
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    '''
    Compute the Hamming score (a.k.a. label-based accuracy) for the multi-label case
    http://stackoverflow.com/q/32239577/395857
    '''
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        #print('\nset_true: {0}'.format(set_true))
        #print('set_pred: {0}'.format(set_pred))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean(acc_list)

In [180]:
def print_statistics(y, y_pred):
    accuracy = hamming_score(y, y_pred)
    precision = metrics.precision_score(y, y_pred, average='weighted')
    recall = metrics.recall_score(y, y_pred, average='weighted')
    f_score = metrics.f1_score(y, y_pred, average='weighted')
    print('Accuracy: %.3f\nPrecision: %.3f\nRecall: %.3f\nF_score: %.3f\n'
          % (accuracy, precision, recall, f_score))
    print(metrics.classification_report(y, y_pred,target_names=['sarcasm', 'irony',
       'satire', 'understatement', 'overstatement', 'rhetorical_question']))
    return accuracy, precision, recall, f_score



def plot_coefficients(classifier, feature_names, top_features=20, plot_name="/bow_models/bow_binary_"):
    # Get the top most positive/negative coefficients
    coef = classifier.coef_.ravel()
    top_positive_coefficients = np.argsort(coef)[-top_features:]
    top_negative_coefficients = np.argsort(coef)[:top_features]
    top_coefficients = np.hstack([top_negative_coefficients, top_positive_coefficients])
    x_names = [feature_names[feature] for feature in top_coefficients]

    # Plot the coefficients
    plt.figure(figsize=(15, 5))
    colors = ['red' if c < 0 else 'blue' for c in coef[top_coefficients]]
    plt.bar(np.arange(2 * top_features), coef[top_coefficients], color=colors)
    plt.xticks(np.arange(0, 2 * top_features), x_names, rotation=30, ha='right')
    plt.ylabel("Coefficient Value")
    plt.title("Visualising the top %d features taken up by an SVM model" % top_features)
    to_save_filename = path + "/plots/" + plot_name + "top%d_coefficients.png" % top_features
    plt.savefig(to_save_filename)
    print("Coefficients' visualisation saved to %s\n" % to_save_filename)

def get_regularization_params(a=-1, b=1, c=3, d=1, e=5):
    reg_range = np.outer(np.logspace(a, b, c), np.array([d, e]))
    reg_range = reg_range.flatten()
    return reg_range


def grid_classifier(x_train, y_train, x_test, y_test, model, parameters,
                    make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    grid = GridSearchCV(estimator=model, param_grid=parameters, verbose=0)
    grid.fit(x_train, y_train)
    sorted(grid.cv_results_.keys())
    classifier = grid.best_estimator_
    if make_feature_analysis:
        plot_coefficients(classifier, feature_names, top_features, plot_name)
    y_hat = classifier.predict(x_test)
    print_statistics(y_test, y_hat)

# Method to print the header of the currently running model
def print_model_title(name):
    print("\n==================================================================")
    print('{:>20}'.format(name))
    print("==================================================================\n")


def linear_svm_grid(x_train, y_train, x_test, y_test, class_ratio,
               make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    print_model_title("Linear SVM")
    C_range = get_regularization_params()
    parameters = {'estimator__C': C_range}
    linear_svm = MultiOutputClassifier(LinearSVC(C=1.0, class_weight=class_ratio, penalty='l2'))
    # print(linear_svm.get_params().keys())
    grid_classifier(x_train, y_train, x_test, y_test, linear_svm, parameters,
                    make_feature_analysis, feature_names, top_features, plot_name)


def nonlinear_svm_grid(x_train, y_train, x_test, y_test, class_ratio,
                  make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    print_model_title("Nonlinear SVM")
    C_range = get_regularization_params(a=-1, b=0, c=2, d=1, e=5)
    gamma_range = get_regularization_params(a=-2, b=-1, c=2, d=1, e=5)
    parameters = {'estimator__kernel': ['rbf'], 'estimator__C': C_range, 'estimator__gamma': gamma_range}
    nonlinear_svm = MultiOutputClassifier(SVC(class_weight=class_ratio))
    grid_classifier(x_train, y_train, x_test, y_test, nonlinear_svm, parameters,
                    make_feature_analysis, feature_names, top_features, plot_name)


def logistic_regression_grid(x_train, y_train, x_test, y_test, class_ratio,
                        make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    print_model_title("Logistic Regression")
    C_range = [0.001, 0.01, 0.1, 1, 10, 100]
    parameters = {'estimator__C': C_range}
    log_regr = MultiOutputClassifier(LogisticRegression(C=1.0, class_weight=class_ratio, penalty='l2'))
    grid_classifier(x_train, y_train, x_test, y_test, log_regr, parameters,
                    make_feature_analysis, feature_names, top_features, plot_name)


def linear_svm(x_train, y_train, x_test, y_test, class_ratio='balanced'):
    print_model_title("Linear SVM")
    svm = MultiOutputClassifier(LinearSVC(C=0.01, class_weight=class_ratio, penalty='l2'))
    svm.fit(x_train, y_train)
    y_hat = svm.predict(x_test)
    print_statistics(y_test, y_hat)
    # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/svm_model_bin.sav'
    # joblib.dump(svm, filename)


def logistic_regression(x_train, y_train, x_test, y_test, class_ratio='balanced'):
    print_model_title("Logistic Regression")
    regr = MultiOutputClassifier(LogisticRegression(C=0.01, class_weight=class_ratio, penalty='l2'))
    regr.fit(x_train, y_train)
    y_hat = regr.predict(x_test)
    print_statistics(y_test, y_hat)
    # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
    # joblib.dump(regr, filename)


def random_forest(x_train, y_train, x_test, y_test, class_ratio='balanced'):
  print_model_title("Random Forest")
  rf = MultiOutputClassifier(RandomForestClassifier(n_estimators=400, random_state=11))
  rf.fit(x_train, y_train)
  y_hat = rf.predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)

def xg_boost(x_train, y_train, x_test, y_test):
  print_model_title("XGBoost")
  xgb_model =MultiOutputClassifier(XGBClassifier(max_depth=6, n_estimators=1000))
  xgb_model .fit(x_train, y_train)
  y_hat = xgb_model .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)


def xg_boost_focal_loss(x_train, y_train, x_test, y_test):
  print_model_title("XGBoost Focal")
  xgboster_focal = imb_xgb(special_objective='focal')
  CV_focal_booster = GridSearchCV(xgboster_focal, {"focal_gamma":[1.0,1.5,2.0,2.5,3.0]})
  CV_focal_booster.fit(x_train, y_train)
  opt_focal_booster = CV_focal_booster.best_estimator_
  # xgb_model .fit(x_train, y_train)
  y_hat = opt_focal_booster.predict_determine(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)

def xg_boost_weighted_loss(x_train, y_train, x_test, y_test):
  print_model_title("XGBoost Weighted")
  xgboster_focal = imb_xgb(special_objective='weighted')
  CV_focal_booster = GridSearchCV(xgboster_focal, {"imbalance_alpha":[1.5,2.0,2.5,3.0,4.0]})
  CV_focal_booster.fit(x_train, y_train)
  opt_focal_booster = CV_focal_booster.best_estimator_
  # xgb_model .fit(x_train, y_train)
  y_hat = opt_focal_booster.predict_determine(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)


def feature_selection(x_train, y_train, x_test, y_test):
    print("Feature selection with LinearSVC")
    model = MultiOutputClassifier(LinearSVC(C=0.1, penalty='l2'))
    rfe = RFE(model, 5)
    best_features_model = rfe.fit(x_train, y_train)
    y_hat = best_features_model.predict(x_test)
    print_statistics(y_test, y_hat)


def ensemble_stacked(x_train, y_train, x_test, y_test):
  print_model_title("Ensemble Stacked Classifiers")
  estimators = [ ('lr',MultiOutputClassifier(LogisticRegression(C=0.01, class_weight='balanced', penalty='l2'))),('svm_linear',MultiOutputClassifier(LinearSVC(C=0.01, class_weight='balanced', penalty='l2'))),('rf', MultiOutputClassifier(RandomForestClassifier(n_estimators=10, random_state=42)))]
  from sklearn.ensemble import StackingClassifier
  clf = StackingClassifier(
      estimators=estimators )
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)



from sklearn.ensemble import RandomForestClassifier, VotingClassifier
def voting_classifiers(x_train, y_train, x_test, y_test,voting_type='hard'):
  print_model_title("Voting Classifier")
  estimators = [ ('lr',MultiOutputClassifier(LogisticRegression(C=0.01, class_weight='balanced', penalty='l2'))),('svm_linear',MultiOutputClassifier(LinearSVC(C=0.01, class_weight='balanced', penalty='l2'))),('rf',MultiOutputClassifier(RandomForestClassifier(n_estimators=10, random_state=42)))]
  from sklearn.ensemble import StackingClassifier
  clf = VotingClassifier(
      estimators=estimators , voting=voting_type)
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)



from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble import BaggingClassifier
def Bagging_Classifier_LR(x_train, y_train, x_test, y_test):
  print_model_title("Bagging Calssifier LR")
 
  clf =BaggingClassifier(base_estimator=MultiOutputClassifier(LogisticRegression(C=0.01, class_weight='balanced', penalty='l2')),
                       n_estimators=10, random_state=42)
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)


def Bagging_Classifier_SVM(x_train, y_train, x_test, y_test):
  print_model_title("Bagging Calssifier SVM")
 
  clf =BaggingClassifier(base_estimator=MultiOutputClassifier(LinearSVC(C=0.01, class_weight='balanced', penalty='l2')),
                       n_estimators=10, random_state=42)
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)


from sklearn.ensemble import GradientBoostingClassifier
def gradient_boosting(x_train, y_train, x_test, y_test):
  print_model_title("Gradient Boosting")
 
  clf =MultiOutputClassifier(GradientBoostingClassifier(n_estimators=100, learning_rate=0.01,max_depth=30, random_state=42))
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)


In [181]:
import warnings
warnings.filterwarnings('ignore')

In [182]:
y_train_l=[]
for i in range(0,len(y_train.to_numpy())):
  y_train_l.append(y_train.to_numpy()[i][0].astype(int))
y_train_l=np.array(y_train_l)


In [183]:
y_train=y_train_l

In [184]:
y_test_l=[]
for i in range(0,len(y_test.to_numpy())):
  y_test_l.append(y_test.to_numpy()[i][0].astype(int))
y_test_l=np.array(y_test_l)

In [185]:
y_test=y_test_l

In [186]:
linear_svm_grid(x_train,y_train,x_test,y_test,class_ratio='balanced')


          Linear SVM

Accuracy: 0.579
Precision: 0.638
Recall: 0.687
F_score: 0.659

                     precision    recall  f1-score   support

            sarcasm       0.81      0.81      0.81       138
              irony       0.27      0.39      0.32        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.54      0.76      0.63        25

          micro avg       0.58      0.69      0.63       211
          macro avg       0.27      0.33      0.29       211
       weighted avg       0.64      0.69      0.66       211
        samples avg       0.62      0.70      0.64       211



In [187]:
nonlinear_svm_grid(x_train,y_train.astype(int),x_test,y_test.astype(int),class_ratio='balanced')


       Nonlinear SVM

Accuracy: 0.707
Precision: 0.725
Recall: 0.659
F_score: 0.596

                     precision    recall  f1-score   support

            sarcasm       0.80      0.99      0.88       138
              irony       0.50      0.03      0.05        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       1.00      0.04      0.08        25

          micro avg       0.79      0.66      0.72       211
          macro avg       0.38      0.18      0.17       211
       weighted avg       0.72      0.66      0.60       211
        samples avg       0.79      0.71      0.74       211



In [188]:
logistic_regression_grid(x_train,y_train.astype(int),x_test,y_test.astype(int),class_ratio='balanced')


 Logistic Regression

Accuracy: 0.316
Precision: 0.675
Recall: 0.493
F_score: 0.508

                     precision    recall  f1-score   support

            sarcasm       0.84      0.41      0.55       138
              irony       0.23      0.67      0.34        36
             satire       0.03      0.20      0.05         5
     understatement       0.01      1.00      0.03         1
      overstatement       0.05      0.50      0.09         6
rhetorical_question       0.73      0.76      0.75        25

          micro avg       0.28      0.49      0.36       211
          macro avg       0.31      0.59      0.30       211
       weighted avg       0.67      0.49      0.51       211
        samples avg       0.34      0.48      0.38       211



In [189]:
linear_svm(x_train,y_train.astype(int),x_test,y_test.astype(int),class_ratio='balanced')


          Linear SVM

Accuracy: 0.303
Precision: 0.651
Recall: 0.536
F_score: 0.538

                     precision    recall  f1-score   support

            sarcasm       0.81      0.48      0.60       138
              irony       0.22      0.58      0.32        36
             satire       0.07      0.60      0.12         5
     understatement       0.01      1.00      0.02         1
      overstatement       0.04      0.50      0.08         6
rhetorical_question       0.66      0.76      0.70        25

          micro avg       0.28      0.54      0.37       211
          macro avg       0.30      0.65      0.31       211
       weighted avg       0.65      0.54      0.54       211
        samples avg       0.32      0.52      0.38       211



In [190]:
logistic_regression(x_train,y_train.astype(int),x_test,y_test.astype(int),class_ratio='balanced')


 Logistic Regression

Accuracy: 0.302
Precision: 0.644
Recall: 0.536
F_score: 0.536

                     precision    recall  f1-score   support

            sarcasm       0.82      0.49      0.61       138
              irony       0.23      0.58      0.33        36
             satire       0.03      0.20      0.05         5
     understatement       0.02      1.00      0.03         1
      overstatement       0.05      0.67      0.09         6
rhetorical_question       0.58      0.76      0.66        25

          micro avg       0.29      0.54      0.38       211
          macro avg       0.29      0.62      0.29       211
       weighted avg       0.64      0.54      0.54       211
        samples avg       0.32      0.53      0.39       211



In [191]:
random_forest(x_train,y_train.astype(int),x_test,y_test.astype(int),class_ratio='balanced')


       Random Forest

Accuracy: 0.717
Precision: 0.640
Recall: 0.673
F_score: 0.613

                     precision    recall  f1-score   support

            sarcasm       0.80      1.00      0.89       138
              irony       0.00      0.00      0.00        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       1.00      0.16      0.28        25

          micro avg       0.80      0.67      0.73       211
          macro avg       0.30      0.19      0.19       211
       weighted avg       0.64      0.67      0.61       211
        samples avg       0.80      0.72      0.74       211



In [192]:
xg_boost(x_train,y_train.astype(int),x_test,y_test.astype(int))


             XGBoost

Accuracy: 0.705
Precision: 0.669
Recall: 0.692
F_score: 0.657

                     precision    recall  f1-score   support

            sarcasm       0.81      0.96      0.88       138
              irony       0.29      0.06      0.09        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.79      0.44      0.56        25

          micro avg       0.77      0.69      0.73       211
          macro avg       0.31      0.24      0.26       211
       weighted avg       0.67      0.69      0.66       211
        samples avg       0.76      0.72      0.73       211



In [194]:
gradient_boosting(x_train,y_train.astype(int),x_test,y_test.astype(int))


   Gradient Boosting

Accuracy: 0.656
Precision: 0.631
Recall: 0.692
F_score: 0.656

                     precision    recall  f1-score   support

            sarcasm       0.80      0.93      0.86       138
              irony       0.19      0.11      0.14        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.67      0.56      0.61        25

          micro avg       0.69      0.69      0.69       211
          macro avg       0.28      0.27      0.27       211
       weighted avg       0.63      0.69      0.66       211
        samples avg       0.70      0.71      0.69       211



In [195]:
word_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')


# Character Stemmer
char_vectorizer = TfidfVectorizer(min_df= 3, max_df=0.5, analyzer = 'char_wb', ngram_range = (3,5))

word_vectorizer.fit(result['demoji_text'])
char_vectorizer.fit(result['demoji_text'])


TfidfVectorizer(analyzer='char_wb', max_df=0.5, min_df=3, ngram_range=(3, 5))

In [196]:
train_word_features = word_vectorizer.transform(result['demoji_text'])
train_char_features = char_vectorizer.transform(result['demoji_text'])

In [197]:
test_word_features = word_vectorizer.transform(result_test['demoji_text'])
test_char_features = char_vectorizer.transform(result_test['demoji_text'])

In [198]:
from scipy.sparse import hstack, csr_matrix


In [199]:
train_features = hstack([
    train_char_features,
    train_word_features
    ,csr_matrix(np.array(x_train.to_numpy(), dtype=float))], 'csr'
)

In [200]:
test_features = hstack([
    test_char_features,
    test_word_features
    ,csr_matrix(np.array(x_test.to_numpy(), dtype=float))], 'csr'
)

In [201]:
linear_svm_grid(train_features,y_train.astype(int),test_features,y_test.astype(int),class_ratio='balanced')


          Linear SVM

Accuracy: 0.669
Precision: 0.694
Recall: 0.668
F_score: 0.679

                     precision    recall  f1-score   support

            sarcasm       0.82      0.81      0.82       138
              irony       0.33      0.36      0.34        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.84      0.64      0.73        25

          micro avg       0.71      0.67      0.69       211
          macro avg       0.33      0.30      0.31       211
       weighted avg       0.69      0.67      0.68       211
        samples avg       0.71      0.70      0.69       211



In [202]:
nonlinear_svm_grid(train_features,y_train.astype(int),test_features,y_test.astype(int),class_ratio='balanced')


       Nonlinear SVM

Accuracy: 0.707
Precision: 0.519
Recall: 0.654
F_score: 0.579

                     precision    recall  f1-score   support

            sarcasm       0.79      1.00      0.88       138
              irony       0.00      0.00      0.00        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.00      0.00      0.00        25

          micro avg       0.79      0.65      0.72       211
          macro avg       0.13      0.17      0.15       211
       weighted avg       0.52      0.65      0.58       211
        samples avg       0.79      0.71      0.74       211



In [203]:
logistic_regression_grid(train_features,y_train.astype(int),test_features,y_test.astype(int),class_ratio='balanced')


 Logistic Regression

Accuracy: 0.670
Precision: 0.705
Recall: 0.664
F_score: 0.675

                     precision    recall  f1-score   support

            sarcasm       0.83      0.83      0.83       138
              irony       0.32      0.36      0.34        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.92      0.48      0.63        25

          micro avg       0.71      0.66      0.69       211
          macro avg       0.34      0.28      0.30       211
       weighted avg       0.70      0.66      0.68       211
        samples avg       0.72      0.69      0.70       211



In [204]:
linear_svm(train_features,y_train.astype(int),test_features,y_test.astype(int),class_ratio='balanced')


          Linear SVM

Accuracy: 0.390
Precision: 0.642
Recall: 0.536
F_score: 0.555

                     precision    recall  f1-score   support

            sarcasm       0.80      0.52      0.63       138
              irony       0.22      0.50      0.31        36
             satire       0.08      0.40      0.14         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.05      0.33      0.09         6
rhetorical_question       0.66      0.76      0.70        25

          micro avg       0.38      0.54      0.45       211
          macro avg       0.30      0.42      0.31       211
       weighted avg       0.64      0.54      0.55       211
        samples avg       0.42      0.52      0.45       211



In [205]:
logistic_regression(train_features,y_train.astype(int),test_features,y_test.astype(int),class_ratio='balanced')


 Logistic Regression

Accuracy: 0.344
Precision: 0.647
Recall: 0.555
F_score: 0.555

                     precision    recall  f1-score   support

            sarcasm       0.82      0.52      0.64       138
              irony       0.23      0.56      0.33        36
             satire       0.03      0.20      0.05         5
     understatement       0.02      1.00      0.04         1
      overstatement       0.05      0.67      0.10         6
rhetorical_question       0.59      0.76      0.67        25

          micro avg       0.32      0.55      0.41       211
          macro avg       0.29      0.62      0.30       211
       weighted avg       0.65      0.55      0.56       211
        samples avg       0.37      0.55      0.42       211



In [206]:
random_forest(train_features,y_train.astype(int),test_features,y_test.astype(int),class_ratio='balanced')



       Random Forest

Accuracy: 0.710
Precision: 0.692
Recall: 0.654
F_score: 0.587

                     precision    recall  f1-score   support

            sarcasm       0.80      0.99      0.88       138
              irony       1.00      0.03      0.05        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.00      0.00      0.00        25

          micro avg       0.80      0.65      0.72       211
          macro avg       0.30      0.17      0.16       211
       weighted avg       0.69      0.65      0.59       211
        samples avg       0.79      0.71      0.74       211



In [207]:
xg_boost(train_features,y_train.astype(int),test_features,y_test.astype(int))



             XGBoost

Accuracy: 0.717
Precision: 0.677
Recall: 0.687
F_score: 0.645

                     precision    recall  f1-score   support

            sarcasm       0.80      0.98      0.88       138
              irony       0.40      0.06      0.10        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.73      0.32      0.44        25

          micro avg       0.78      0.69      0.73       211
          macro avg       0.32      0.23      0.24       211
       weighted avg       0.68      0.69      0.64       211
        samples avg       0.78      0.73      0.74       211



In [208]:
gradient_boosting(train_features,y_train.astype(int),test_features,y_test.astype(int))


   Gradient Boosting

Accuracy: 0.684
Precision: 0.603
Recall: 0.692
F_score: 0.636

                     precision    recall  f1-score   support

            sarcasm       0.79      0.97      0.87       138
              irony       0.11      0.03      0.04        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.55      0.44      0.49        25

          micro avg       0.72      0.69      0.70       211
          macro avg       0.24      0.24      0.23       211
       weighted avg       0.60      0.69      0.64       211
        samples avg       0.74      0.72      0.72       211



In [209]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1,1),
    dtype=np.float32,
    max_features=5000
)


# Character Stemmer
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(1, 4),
    dtype=np.float32,
    max_features=4000
)

word_vectorizer.fit(result['demoji_text'])
char_vectorizer.fit(result['demoji_text'])


TfidfVectorizer(analyzer='char', dtype=<class 'numpy.float32'>,
                max_features=4000, ngram_range=(1, 4), strip_accents='unicode',
                sublinear_tf=True)

In [210]:
train_word_features = word_vectorizer.transform(result['demoji_text'])
train_char_features = char_vectorizer.transform(result['demoji_text'])

In [211]:
test_word_features = word_vectorizer.transform(result_test['demoji_text'])
test_char_features = char_vectorizer.transform(result_test['demoji_text'])

In [212]:
from scipy.sparse import hstack, csr_matrix


In [213]:
train_features = hstack([
    train_char_features,
    train_word_features
    ,csr_matrix(np.array(x_train.to_numpy(), dtype=float))], 'csr'
)

In [214]:
test_features = hstack([
    test_char_features,
    test_word_features
    ,csr_matrix(np.array(x_test.to_numpy(), dtype=float))], 'csr'
)

In [215]:
linear_svm_grid(train_features,y_train.astype(int),test_features,y_test.astype(int),class_ratio='balanced')


          Linear SVM

Accuracy: 0.668
Precision: 0.681
Recall: 0.664
F_score: 0.665

                     precision    recall  f1-score   support

            sarcasm       0.80      0.86      0.83       138
              irony       0.27      0.22      0.24        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.93      0.52      0.67        25

          micro avg       0.72      0.66      0.69       211
          macro avg       0.33      0.27      0.29       211
       weighted avg       0.68      0.66      0.66       211
        samples avg       0.71      0.69      0.69       211



In [216]:
nonlinear_svm_grid(train_features,y_train.astype(int),test_features,y_test.astype(int),class_ratio='balanced')


       Nonlinear SVM

Accuracy: 0.707
Precision: 0.519
Recall: 0.654
F_score: 0.579

                     precision    recall  f1-score   support

            sarcasm       0.79      1.00      0.88       138
              irony       0.00      0.00      0.00        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.00      0.00      0.00        25

          micro avg       0.79      0.65      0.72       211
          macro avg       0.13      0.17      0.15       211
       weighted avg       0.52      0.65      0.58       211
        samples avg       0.79      0.71      0.74       211



In [217]:
logistic_regression_grid(train_features,y_train.astype(int),test_features,y_test.astype(int),class_ratio='balanced')


 Logistic Regression

Accuracy: 0.658
Precision: 0.683
Recall: 0.654
F_score: 0.659

                     precision    recall  f1-score   support

            sarcasm       0.81      0.86      0.83       138
              irony       0.21      0.19      0.20        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       1.00      0.52      0.68        25

          micro avg       0.71      0.65      0.68       211
          macro avg       0.34      0.26      0.29       211
       weighted avg       0.68      0.65      0.66       211
        samples avg       0.70      0.68      0.68       211



In [218]:
linear_svm(train_features,y_train.astype(int),test_features,y_test.astype(int),class_ratio='balanced')


          Linear SVM

Accuracy: 0.402
Precision: 0.647
Recall: 0.559
F_score: 0.567

                     precision    recall  f1-score   support

            sarcasm       0.81      0.54      0.65       138
              irony       0.22      0.50      0.31        36
             satire       0.08      0.40      0.13         5
     understatement       0.03      1.00      0.06         1
      overstatement       0.07      0.50      0.12         6
rhetorical_question       0.66      0.76      0.70        25

          micro avg       0.39      0.56      0.46       211
          macro avg       0.31      0.62      0.33       211
       weighted avg       0.65      0.56      0.57       211
        samples avg       0.43      0.55      0.46       211



In [219]:
logistic_regression(train_features,y_train.astype(int),test_features,y_test.astype(int),class_ratio='balanced')


 Logistic Regression

Accuracy: 0.340
Precision: 0.644
Recall: 0.555
F_score: 0.555

                     precision    recall  f1-score   support

            sarcasm       0.81      0.52      0.63       138
              irony       0.23      0.56      0.33        36
             satire       0.03      0.20      0.05         5
     understatement       0.02      1.00      0.04         1
      overstatement       0.05      0.67      0.09         6
rhetorical_question       0.61      0.76      0.68        25

          micro avg       0.32      0.55      0.41       211
          macro avg       0.29      0.62      0.30       211
       weighted avg       0.64      0.55      0.56       211
        samples avg       0.37      0.55      0.42       211



In [220]:
random_forest(train_features,y_train.astype(int),test_features,y_test.astype(int),class_ratio='balanced')



       Random Forest

Accuracy: 0.707
Precision: 0.519
Recall: 0.654
F_score: 0.579

                     precision    recall  f1-score   support

            sarcasm       0.79      1.00      0.88       138
              irony       0.00      0.00      0.00        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.00      0.00      0.00        25

          micro avg       0.79      0.65      0.72       211
          macro avg       0.13      0.17      0.15       211
       weighted avg       0.52      0.65      0.58       211
        samples avg       0.79      0.71      0.74       211



In [221]:
xg_boost(train_features,y_train.astype(int),test_features,y_test.astype(int))



             XGBoost

Accuracy: 0.739
Precision: 0.710
Recall: 0.720
F_score: 0.666

                     precision    recall  f1-score   support

            sarcasm       0.80      1.00      0.89       138
              irony       0.50      0.03      0.05        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.87      0.52      0.65        25

          micro avg       0.80      0.72      0.76       211
          macro avg       0.36      0.26      0.27       211
       weighted avg       0.71      0.72      0.67       211
        samples avg       0.80      0.75      0.76       211



In [222]:
gradient_boosting(train_features,y_train.astype(int),test_features,y_test.astype(int))


   Gradient Boosting

Accuracy: 0.656
Precision: 0.639
Recall: 0.668
F_score: 0.644

                     precision    recall  f1-score   support

            sarcasm       0.79      0.91      0.85       138
              irony       0.25      0.14      0.18        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.25      0.17      0.20         6
rhetorical_question       0.60      0.36      0.45        25

          micro avg       0.71      0.67      0.69       211
          macro avg       0.32      0.26      0.28       211
       weighted avg       0.64      0.67      0.64       211
        samples avg       0.71      0.70      0.69       211



In [223]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1,1),
    dtype=np.float32,
    max_features=5000
)


# Character Stemmer
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(1, 4),
    dtype=np.float32,
    max_features=4000
)

word_vectorizer.fit(result['text'])
char_vectorizer.fit(result['text'])


TfidfVectorizer(analyzer='char', dtype=<class 'numpy.float32'>,
                max_features=4000, ngram_range=(1, 4), strip_accents='unicode',
                sublinear_tf=True)

In [224]:
train_word_features = word_vectorizer.transform(result['text'])
train_char_features = char_vectorizer.transform(result['text'])

In [225]:
test_word_features = word_vectorizer.transform(result_test['text'])
test_char_features = char_vectorizer.transform(result_test['text'])

In [226]:
from scipy.sparse import hstack, csr_matrix


In [227]:
train_features = hstack([
    train_char_features,
    train_word_features
    ,csr_matrix(np.array(x_train.to_numpy(), dtype=float))], 'csr'
)

In [228]:
test_features = hstack([
    test_char_features,
    test_word_features
    ,csr_matrix(np.array(x_test.to_numpy(), dtype=float))], 'csr'
)

In [229]:
linear_svm_grid(train_features,y_train.astype(int),test_features,y_test.astype(int),class_ratio='balanced')


          Linear SVM

Accuracy: 0.679
Precision: 0.690
Recall: 0.659
F_score: 0.658

                     precision    recall  f1-score   support

            sarcasm       0.80      0.88      0.84       138
              irony       0.27      0.17      0.21        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       1.00      0.44      0.61        25

          micro avg       0.74      0.66      0.70       211
          macro avg       0.35      0.25      0.28       211
       weighted avg       0.69      0.66      0.66       211
        samples avg       0.73      0.68      0.70       211



In [230]:
nonlinear_svm_grid(train_features,y_train.astype(int),test_features,y_test.astype(int),class_ratio='balanced')


       Nonlinear SVM

Accuracy: 0.707
Precision: 0.519
Recall: 0.654
F_score: 0.579

                     precision    recall  f1-score   support

            sarcasm       0.79      1.00      0.88       138
              irony       0.00      0.00      0.00        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.00      0.00      0.00        25

          micro avg       0.79      0.65      0.72       211
          macro avg       0.13      0.17      0.15       211
       weighted avg       0.52      0.65      0.58       211
        samples avg       0.79      0.71      0.74       211



In [231]:
logistic_regression_grid(train_features,y_train.astype(int),test_features,y_test.astype(int),class_ratio='balanced')


 Logistic Regression

Accuracy: 0.647
Precision: 0.669
Recall: 0.640
F_score: 0.647

                     precision    recall  f1-score   support

            sarcasm       0.79      0.83      0.81       138
              irony       0.23      0.19      0.21        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.93      0.52      0.67        25

          micro avg       0.70      0.64      0.67       211
          macro avg       0.33      0.26      0.28       211
       weighted avg       0.67      0.64      0.65       211
        samples avg       0.69      0.67      0.67       211



In [232]:
linear_svm(train_features,y_train.astype(int),test_features,y_test.astype(int),class_ratio='balanced')


          Linear SVM

Accuracy: 0.403
Precision: 0.649
Recall: 0.564
F_score: 0.574

                     precision    recall  f1-score   support

            sarcasm       0.81      0.56      0.66       138
              irony       0.23      0.50      0.31        36
             satire       0.04      0.20      0.07         5
     understatement       0.03      1.00      0.06         1
      overstatement       0.07      0.50      0.12         6
rhetorical_question       0.66      0.76      0.70        25

          micro avg       0.39      0.56      0.46       211
          macro avg       0.31      0.59      0.32       211
       weighted avg       0.65      0.56      0.57       211
        samples avg       0.43      0.55      0.46       211



In [233]:
logistic_regression(train_features,y_train.astype(int),test_features,y_test.astype(int),class_ratio='balanced')


 Logistic Regression

Accuracy: 0.340
Precision: 0.643
Recall: 0.550
F_score: 0.552

                     precision    recall  f1-score   support

            sarcasm       0.81      0.51      0.63       138
              irony       0.24      0.56      0.33        36
             satire       0.03      0.20      0.05         5
     understatement       0.02      1.00      0.04         1
      overstatement       0.05      0.67      0.09         6
rhetorical_question       0.61      0.76      0.68        25

          micro avg       0.32      0.55      0.40       211
          macro avg       0.29      0.62      0.30       211
       weighted avg       0.64      0.55      0.55       211
        samples avg       0.37      0.54      0.42       211



In [234]:
random_forest(train_features,y_train.astype(int),test_features,y_test.astype(int),class_ratio='balanced')



       Random Forest

Accuracy: 0.707
Precision: 0.519
Recall: 0.654
F_score: 0.579

                     precision    recall  f1-score   support

            sarcasm       0.79      1.00      0.88       138
              irony       0.00      0.00      0.00        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.00      0.00      0.00        25

          micro avg       0.79      0.65      0.72       211
          macro avg       0.13      0.17      0.15       211
       weighted avg       0.52      0.65      0.58       211
        samples avg       0.79      0.71      0.74       211



In [235]:
xg_boost(train_features,y_train.astype(int),test_features,y_test.astype(int))



             XGBoost

Accuracy: 0.747
Precision: 0.746
Recall: 0.730
F_score: 0.681

                     precision    recall  f1-score   support

            sarcasm       0.80      1.00      0.89       138
              irony       0.67      0.06      0.10        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.93      0.56      0.70        25

          micro avg       0.81      0.73      0.77       211
          macro avg       0.40      0.27      0.28       211
       weighted avg       0.75      0.73      0.68       211
        samples avg       0.80      0.76      0.77       211



In [236]:
gradient_boosting(train_features,y_train.astype(int),test_features,y_test.astype(int))


   Gradient Boosting

Accuracy: 0.672
Precision: 0.648
Recall: 0.673
F_score: 0.646

                     precision    recall  f1-score   support

            sarcasm       0.80      0.93      0.86       138
              irony       0.38      0.14      0.20        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.50      0.36      0.42        25

          micro avg       0.74      0.67      0.70       211
          macro avg       0.28      0.24      0.25       211
       weighted avg       0.65      0.67      0.65       211
        samples avg       0.73      0.70      0.70       211

